In [3]:
!pip install --force-reinstall typing_extensions
!pip install gpustat

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gpustat: filename=gpustat-1.1.1-py3-none-any.whl size=26608 sha256=eb5f8c80da52affc8053f94ed5863f114d3c4ea3fef748d13fd92f03fb913a1b
  Stored in directory: /root/.cache/pip/wheels/c9/2b/d9/a0b77d6e8623ce6b5c73813af455a3ace394abfc2e8aef7ed6
Successfully built gpustat

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import torch
import transformers
import trl

In [2]:
from huggingface_hub import login

login(
  token="key",
  add_to_git_credential=True
)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [4]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    #load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
tokenizer.eos_token

'<eos>'

In [6]:
tokenizer.padding_side

'left'

# 데이터

In [7]:
!wget https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
with open('./total_kor_multiturn_counsel_bot.jsonl', 
          'r', 
          encoding='utf-8') as file:
    original_jsonl_data = [json.loads(line) for line in file]
print(len(original_jsonl_data))
original_jsonl_data[0]


--2025-01-09 09:02:28--  https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30560672 (29M) [application/octet-stream]
Saving to: ‘total_kor_multiturn_counsel_bot.jsonl.1’

total_kor_multiturn 100%[===================>]  29.14M  13.5MB/s    in 2.2s    

2025-01-09 09:02:30 (13.5 MB/s) - ‘total_kor_multiturn_counsel_bot.jsonl.1’ saved [30560672/30560672]

8731


[{'speaker': '상담사', 'utterance': '안녕하세요. 상담사입니다. 무엇이 불편하시나요?'},
 {'speaker': '내담자', 'utterance': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
 {'speaker': '상담사', 'utterance': '중2병 증상이라니, 어떤 증상이신 건가요?'},
 {'speaker': '내담자',
  'utterance': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
 {'speaker': '상담사',
  'utterance': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
 {'speaker': '내담자',
  'utterance': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
 {'speaker': '상담사',
  'utterance': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
 {'speaker': '내담자', 'utterance': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
 {'speaker': '상담사', 'utterance': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
 {'speaker': '내담자',
  'utterance': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
 {'speaker': '상담사',
  'utterance': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
 {'speaker': '내담자',
  'utterance': '좋은 질문이에요. 좀

In [8]:
speaker_dict = {'내담자': 'user', '상담사': 'assistant'}

def preprocess_conversation(messages):
    # speaker를 role로 변환
    converted_messages = [{'role': speaker_dict[m['speaker']], 'content': m['utterance']} for m in messages]
    
    # assistant로 시작하는 경우 첫 메시지 제거
    if converted_messages and converted_messages[0]['role'] == 'assistant':
        converted_messages = converted_messages[1:]
    
    # user로 끝나는 경우 마지막 메시지들 제거
    while converted_messages and converted_messages[-1]['role'] == 'user':
        converted_messages = converted_messages[:-1]
    
    # 연속된 동일 역할의 메시지 병합
    converted_messages = merge_consecutive_messages(converted_messages)
    
    # 대화가 비어있거나 홀수 개의 메시지만 남은 경우 처리
    if not converted_messages or len(converted_messages) % 2 != 0:
        return []
    
    return converted_messages

def merge_consecutive_messages(messages):
    if not messages:
        return []
    
    merged = []
    current_role = messages[0]['role']
    current_content = messages[0]['content']
    
    for message in messages[1:]:
        if message['role'] == current_role:
            current_content += " " + message['content']
        else:
            merged.append({'role': current_role, 'content': current_content})
            current_role = message['role']
            current_content = message['content']
    
    merged.append({'role': current_role, 'content': current_content})
    return merged


def transform_to_new_format(original_data):
    transformed_data = []
    for conversation in original_data:
        processed_conversation = preprocess_conversation(conversation)
        if processed_conversation:
            transformed_data.append(processed_conversation)
    return transformed_data

result = transform_to_new_format(original_jsonl_data)
with open("./train_dataset.jsonl", "w", encoding="utf-8") as file:
    for conversation in result:
        json_obj = {"messages": conversation}
        json.dump(json_obj, file, ensure_ascii=False)
        file.write("\n") 
dataset = load_dataset("json", data_files="./train_dataset.jsonl")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 8731
    })
})

In [9]:
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

In [10]:
train_dataset = dataset["train"].select(range(2000))
test_dataset = dataset["test"].select(range(20))

# 학습전

# 학습

In [9]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./model_output", 
    num_train_epochs=2,          
    per_device_train_batch_size=8,
    gradient_accumulation_steps=5,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=100,            
    eval_strategy="steps",
    eval_steps=100,
    save_steps = 100,
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    #push_to_hub=True,             
    report_to="wandb",            
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
)
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: wooseoksong (wooseoksong-study). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 d

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [10]:
del model
del trainer
torch.cuda.empty_cache()

# 학습 후 추론

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer, 
        StoppingCriteria, 
        StoppingCriteriaList
        )

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

#model_name = "./model_output"
#model_name = "multi_gpu_train/model_output"
model_name = "google/gemma-2-9b-it"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16,
                                            #load_in_8bit = True
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 'user' 토큰의 ID를 찾습니다
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

# 입력 텍스트를 토큰화합니다
input_text = "안녕"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<bos>안녕하세요! 궁금한 점이 있으면


In [2]:

output = model.generate(
    input_ids, 
    max_new_tokens=512,
    #do_sample=True,
    #temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], 
                                  #skip_special_tokens=True
                                 )
print(generated_text)

<bos>안녕하세요! 

오늘은 **"한국어로 맛있는 음식을 표현하는 다양한 방법"**에 대해 알아보겠습니다. 

단순히 "맛있다"라는 단어만으로는 표현할 수 없는 음식의 맛과 느낌을 더욱 풍부하게 전달하는 방법들을 알려드릴게요.

**1. 맛의 종류를 구체적으로 표현하기**

* **달콤함:** 달콤하다, 단맛이 진하다, 꿀처럼 달콤하다
* **신맛:** 신하다, 시큼하다, 상큼하다
* **짠맛:** 짭짤하다, 짠맛이 강하다, 짭조름하다
* **매운맛:** 매운다, 맵다, 뜨겁다, 얼굴이 타듯 매운다
* **고소함:** 고소하다, 견과류처럼 고소하다, 깊은 맛이 있다
* **쓴맛:** 쓴다, 쌉쌀하다, 쌉싸름하다

**2. 식감을 표현하기**

* **부드러움:** 부드럽다, 촉촉하다,  melt-in-your-mouth (입안에서 녹는다)
* **쫄깃함:** 쫄깃하다, 탱탱하다, 쫀득하다
* **크리미함:** 크림이 흐르듯 부드럽다, 부드럽고 촉촉하다
* **바삭함:** 바삭하다, 터지는 맛, 바삭바삭하다

**3. 향을 표현하기**

* **달콤한 향:** 달콤한 향이 난다, 꽃향기가 나는
* **고소한 향:** 고소한 향이 풍부하다, 볶은 향이 난다
* **새콤달콤한 향:** 새콤달콤한 향이 맡는다, 상큼한 향이 난다
* **매운 향:** 매운 향이 난다, 뜨거운 향이 난다

**4. 감정을 표현하기**

* **기분 좋은 맛:** 맛있다, 즐겁다, 행복하다
* **입맛을 돋우는 맛:** 입맛


In [1]:
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer, 
        StoppingCriteria, 
        StoppingCriteriaList
        )

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

#model_name = "./model_output"
model_name = "multi_gpu_train/model_output"
#model_name = "google/gemma-2-9b-it"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16,
                                            #load_in_8bit = True
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# 'user' 토큰의 ID를 찾습니다
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

# 입력 텍스트를 토큰화합니다
input_text = "안녕"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

output = model.generate(
    input_ids, 
    max_new_tokens=512,
    #do_sample=True,
    #temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], 
                                  #skip_special_tokens=True
                                 )
print(generated_text)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

In [ ]:

output = model.generate(
    input_ids, 
    max_new_tokens=512,
    #do_sample=True,
    #temperature=0.7,
    #stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], 
                                  #skip_special_tokens=True
                                 )
print(generated_text)